In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    purge(Model)
db.session.commit()

In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df

,id,username,email,password_hash,about_me,last_seen


In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [9]:
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df


,id,username,email,password_hash,about_me,last_seen
0,1,miguel,miguel@gmail.com,pbkdf2:sha256:150000$VxUquZ8R$7503d8f8da48c42e...,None,2020-03-07 16:40:55.394737


In [10]:
r.headers['location']

'http://127.0.0.1:5000/api/user/1'

In [11]:
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })

    r = requests.post(url, data=payload, headers=headers)
#     print(vars(r))

In [12]:
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,miguel,miguel@gmail.com,pbkdf2:sha256:150000$VxUquZ8R$7503d8f8da48c42e...,None,2020-03-07 16:40:55.394737
1,2,TPN,tpn@example.com,pbkdf2:sha256:150000$T8eoZB0G$5ee8b389ee3a4af8...,None,2020-03-07 16:40:56.477407
2,3,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$ligwM9Ib$b175a1a0bd954d00...,None,2020-03-07 16:40:56.561849
3,4,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$57Vqgaj4$2d6abcf0c5046f28...,None,2020-03-07 16:40:56.644261
4,5,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$paT4gyMp$37d5225143fdbbba...,None,2020-03-07 16:40:56.730165
5,6,Vik,vik@example.com,pbkdf2:sha256:150000$PFU5AH4D$2cf46e78fd9419d4...,None,2020-03-07 16:40:56.818769
6,7,Kevin,kevin@example.com,pbkdf2:sha256:150000$nsA46FTl$b20fff452d9bd87d...,None,2020-03-07 16:40:56.903089
7,8,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$SRwSx9xv$b82cadb70d606626...,None,2020-03-07 16:40:56.985654
8,9,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$TSKST08t$eeefc01c06937286...,None,2020-03-07 16:40:57.073032


In [13]:
# whenever a user registers, the response contains the url to that user's homepage
r.headers['location']

'http://127.0.0.1:5000/api/user/9'

In [14]:
# login with miguel credentials
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/login

url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=('miguel', 'python'))

In [15]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [16]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))

In [17]:
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4MzU5OTI1OSwiZXhwIjoxNTgzNTk5ODU5fQ.eyJpZCI6MX0.APVhXqPT4rwRH2OTQnudH0u8JeZy3Nv4DaZmxqeuXKcphxrrplrQH_srP9YZ_-yDxyOcFZa_f5gZfF6GcUXQhw'

In [18]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [19]:
vars(r)

{'_content': b'{"data":"Hello, miguel!"}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Content-Type': 'application/json', 'Content-Length': '26', 'Server': 'Werkzeug/1.0.0 Python/3.6.9', 'Date': 'Sat, 07 Mar 2020 16:40:59 GMT'},
 'raw': <urllib3.response.HTTPResponse at 0x7fba8aadc898>,
 'url': 'http://127.0.0.1:5000/api/login',
 'encoding': None,
 'history': [],
 'reason': 'OK',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(0, 0, 4271),
 'request': <PreparedRequest [GET]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7fba8aadc048>}

In [20]:
# now this user will create a tournament

In [21]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [22]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [23]:
# this our bracket (input to bracket creation):
tuple_list

[('Ptolemy', 8),
 ('Kevin', 3),
 ('Vik', 5),
 ('Sunrisebanana', 1),
 ('Burnaby', 6),
 ('AngryFalco', 2),
 ('Spaceghost', 7),
 ('TPN', 4)]

In [24]:
# create a json payload from the tuple list
url = 'http://127.0.0.1:5000/api/created-tournaments/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "users":usernames,
    "seeds":seeds,
    "tournament_name":"Fall Charity LAN 2018 Melee Singles"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))

In [25]:
btype = BracketTypes.DOUBLE_ELIMINATION
vars(btype)

{'_value_': 1,
 '_name_': 'DOUBLE_ELIMINATION',
 '__objclass__': <enum 'BracketTypes'>}

In [26]:
t = Tournament(tuple_list, btype)

4


In [27]:
q = \
"""
    select * from match where user_1 and user_2
"""

In [28]:
df = pd.read_sql_query(q, cnx)
df

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,7106b296-5b66-4d64-bfe6-76c8d9ae5fee,4,None,5,None,None,1,5,10
1,2,48dbf3bb-1976-478e-80ce-20473308e8d6,3,None,8,None,None,1,5,10
2,3,034078de-e5f6-469c-8b05-c10c3f6118ac,7,None,9,None,None,1,6,11
3,4,f1aeae44-d8ef-4ded-a1cd-c493173942fe,2,None,6,None,None,1,6,11


In [29]:
active_match_ids = df.id.values
active_match_ids

array([1, 2, 3, 4])

In [30]:
for i in active_match_ids:
    user_ids = [
        df.loc[df['id'] == i].user_1.values[0],
        df.loc[df['id'] == i].user_2.values[0]
    ]
    scores = random.sample(range(5), 2)
    w_id = user_ids[np.argmax(scores)]
    l_id = user_ids[np.argmin(scores)]
#     print(w_id)
#     print(l_id)
    payload = json.dumps({
        "entrant1_score" : str(scores[0]),
        "entrant2_score" : str(scores[1]),
        "winner" : str(w_id),
        "loser" : str(l_id)
    })
    url = f'http://127.0.0.1:5000/api/match/{i}/report-match'
    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
    r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
    print(r.content)
    break
    

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'


In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "entrant1_score":4,
    "entrant2_score":3,
    "winner":,
    "loser":,
})

In [11]:

TO = t.bracket.entrants[0]

# initial post to db
t.post_to_db(tournament_name, TO,)

In [12]:
# post self references in matches separately
for r in t.bracket.rounds:
    for m in r.matches:
        m.post_self_refs()

In [13]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,TPN,tpn@example.com,None,None,2020-02-20 01:20:58.042862
1,2,AngryFalco,angryfalco@example.com,None,None,2020-02-20 01:20:58.043367
2,3,Sunrisebanana,sunrisebanana@example.com,None,None,2020-02-20 01:20:58.043462
3,4,Ptolemy,ptolemy@example.com,None,None,2020-02-20 01:20:58.043541
4,5,Vik,vik@example.com,None,None,2020-02-20 01:20:58.043612
5,6,Kevin,kevin@example.com,None,None,2020-02-20 01:20:58.043696
6,7,Spaceghost,spaceghost@example.com,None,None,2020-02-20 01:20:58.043772
7,8,Burnaby,burnaby@example.com,None,None,2020-02-20 01:20:58.043859


In [14]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id
0,1,8,Fall Charity LAN 2018 Melee Singles,2


In [15]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [16]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,2,1
1,8,1
2,5,1
3,3,1
4,6,1
5,7,1
6,1,1
7,4,1


In [17]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [18]:
df = pd.read_sql_query('select * from match', cnx)
df
# df.loc[df['uuid'] == '94de92a2-13ac-4e9a-9bf2-78636100a7b1']

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,0f9a1604-97d0-4a42-b510-56f7ac0f5ae6,2.0,None,4.0,None,None,1,5.0,10.0
1,2,f4f7edfb-9d3b-4348-88a6-cf4dc2eb7cf2,8.0,None,1.0,None,None,1,5.0,10.0
2,3,e3f3fe1d-4907-45e0-a70e-57b32aebec1e,5.0,None,7.0,None,None,1,6.0,11.0
3,4,d2785cd6-9819-4005-8404-b3cd4059e164,3.0,None,6.0,None,None,1,6.0,11.0
4,5,f31792a1-2cc5-4b81-9771-3f0d539c0734,NaN,None,NaN,None,None,2,7.0,12.0
5,6,f8cf1b92-7bb5-4a74-b88d-7e9dd78217f5,NaN,None,NaN,None,None,2,7.0,13.0
6,7,ec8fb7c6-f4bd-4366-a720-d1470479b824,NaN,None,NaN,None,None,3,8.0,15.0
7,8,80667cdb-737f-45e4-ad90-e9cf472506df,NaN,None,NaN,None,None,4,NaN,NaN
8,9,e2e48a25-f7e9-4ada-9dc0-b6a1e889aa36,NaN,None,NaN,None,None,5,NaN,NaN
9,10,e3f465e6-eb1a-4e62-9537-dfa1aaf32132,NaN,None,NaN,None,None,6,12.0,NaN
